In [1]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
@tool
def multiply(a: int, b: int)->int:
    '''Multiply 2 numbers'''
    return a*b

In [3]:
multiply.description

'Multiply 2 numbers'

In [4]:
multiply.invoke({'a':10, "b":9})

90

In [5]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [8]:
# now add toolkit to the llms

In [10]:
llm_tools = llm.bind_tools([multiply])

In [13]:
ans = llm_tools.invoke('what is the multiply 2 with 3')

In [15]:
ans.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.0, 'b': 3.0},
  'id': 'de246069-61af-426b-b3bf-e8502d0e2823',
  'type': 'tool_call'}]

In [17]:
multiply.invoke(ans.tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='de246069-61af-426b-b3bf-e8502d0e2823')

## Make Currency converter

In [198]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests
import json

In [20]:
# making of tools
# 1. get current currency rate of that country
# 2. change from one to another

In [152]:
@tool 
def get_conversion_factor(base_currency: str, target_currency: str)->float:
    '''This function fetches the currency conversion factor between a given base currency and a target currency'''

    url = f'https://v6.exchangerate-api.com/v6/e28b8e23aa04d56a1f676e51/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg])->float:
    '''given a currency conversion rate this function calculates the target currency value from a given base currency value'''

    return base_currency_value * conversion_rate

In [153]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757376001,
 'time_last_update_utc': 'Tue, 09 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1757462401,
 'time_next_update_utc': 'Wed, 10 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.0852}

In [154]:
convert.invoke({'base_currency_value':10, "conversion_rate":111})

1110.0

In [185]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
model_tools = model.bind_tools([get_conversion_factor, convert])

In [199]:
# message --> stroe the conversion
message = []


In [200]:
quey = HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')
message.append(quey)

In [201]:
message

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [202]:
response = model_tools.invoke(message)

In [212]:
response.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '095592f0-462b-40e2-90b7-930f6e0c4fd1',
  'type': 'tool_call'}]

In [204]:
message.append(response)
message

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--2af4ec2a-85c2-4a8c-85ef-49a8c161d004-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '095592f0-462b-40e2-90b7-930f6e0c4fd1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 26, 'total_tokens': 432, 'input_token_details': {'cache_read': 0}})]

In [ ]:
# 1 tool call 
tool_message1 = get_conversion_factor.invoke(response.tool_calls[0])
message.append(tool_message1)
conversion_rate = json.loads(tool_message1.content)['conversion_rate']



# 2 tool call


In [ ]:
# model invoke 
response2 = model_tools.invoke(message)
response2

AIMessage(content='The conversion factor between USD and INR is 88.0852.\nBased on this, 10 USD is equal to 880.852 INR.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--5d0ffd3b-d61e-4d4f-a89e-5a3cb45ceaed-0', usage_metadata={'input_tokens': 376, 'output_tokens': 39, 'total_tokens': 785, 'input_token_details': {'cache_read': 0}})

In [215]:
response2.content

'The conversion factor between USD and INR is 88.0852.\nBased on this, 10 USD is equal to 880.852 INR.'

In [213]:
message.append(response2)

In [214]:
message

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--2af4ec2a-85c2-4a8c-85ef-49a8c161d004-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '095592f0-462b-40e2-90b7-930f6e0c4fd1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 26, 'total_tokens': 432, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1757376